In [1]:
from coppertop.core import *
from coppertop.pipe import *
from dm.examples.cluedo.cards import *
from dm.examples.cluedo.core import one
from dm.examples.cluedo.algos import createBag, figureKnown, processResponses, cluedo_bag, processSuggestions
from dm.examples.cluedo.reports import rep1
from dm.examples.cluedo.core import cluedo_bag, YES, NO, MAYBE, TBI, HasOne

from dm.core import to, each, joinAll, keys, append, select, count, combinations, nCombinations, combinationsR, \
    nCombinationsR, buffer, divide, sum, drop, nPermutations, PP
from dm.examples.cluedo.utils import minus, countIf
from ribs.types import pytuple

__ = Scope()

#### CLUEDO - THE GAME

There are three sets of cards representing the who, with what and where of a murder. One is taken from Players move around a board


#### MODELLING THE PAD

Much of the information leads to mechanical deduction rather than probabilistic deduction. Many of the probabilities 
we think of are uncondictional rather than conditional. 

I can think of a few things we can model - the probability of which hand a card must be in must sum to one. However because of the way the cards are shuffled there are twists - e.g. in the TBI hand adds a thread which run accross all the cards and although we can find PMFs we have to think deeper to find independent PMF.


I've implemented a combination PMF of a hand - as to the observer all possibilities are equally likely - actuall not true a hand with 6 suspects or weapons is impossible as one must be in the TBI.

So the only true model is the arrangements when 1 from each group is given to TBI - and each hand has combinations of the remainder - so (1 out of 6) x (1 out of 6) x (1 out of 9) x C(numCard, 5+5+8)


In [2]:
# total combinations - 250 billion
# 21 card less 3 for TBI -> 18, 4 hands -> 5, 5, 4, 4
324 * (18 >> nCombinations >> 5) * (13 >> nCombinations >> 5) * (8 >> nCombinations >> 4) * (4 >> nCombinations >> 4)

250092722880

In [3]:
# combinations less my hand of 5 cards - 29 million
known = 5 + 1
left = 6+6+9 - 324
324 * (13 >> nCombinations >> 5) * (8 >> nCombinations >> 4) * (4 >> nCombinations >> 4) >> PP
(13 >> nCombinations >> 5) >> PP
(13 >> nCombinations >> 4) >> PP;

29189160
1287
715


In [4]:
21 - 5 >> nCombinations >> 5, (21 - 5 - 3 >> nCombinations >> 5) * 6 * 6 * 9

(4368, 416988)

In [5]:
18 >> nCombinations >> 5, 6 * 6 * 9

(8568, 324)

In [6]:
21 >> nCombinations >> 3

1330

In [7]:
(18 >> nPermutations >> 18) * 324
250_092_722_880
222_713_620_919_193_600
2_074_369_080_655_872_000

2074369080655872000

#### TODO
- display counts for likelihood after display the table - i.e. how many suggestions are made inc own cards, by round number?
- calculate number of combinations for the whole model - reduce down as stuff gets known
- display combination count and unknown card count for each player
- show prior & posterior

- make PP a coppertop fn
- pmfs of prior
- show my info then public info for each player
- sum count for likelihood them over all the games in the sheet
- do updates for individual player models
- figure how to do update for global model

- if someone makes an suggstion - no responses + no accusation -> they must have one or more of the suggestion cards themselves.

In [8]:
@coppertop
def PP(bag:cluedo_bag) -> cluedo_bag:
    bag.pad >> rep1(_, bag.handId) >> PP
    return bag

In [13]:
Me = Pl
events = [
    
    [Me, Gr, Re, Ba], [Gr, Ba],
    [Gr, Mu, Re, Ki], Or - one,
    [Or, Or, Ro, Lo], Pe - one,
    [Pe, Gr, Da, Co], [Me, Da],
    
    [Me, Or, Wr, Co], Gr, Or, [Pe, Or],
    [Gr, Sc, Ca, Ki], Or, Pe, [Me, Ca],
    [Or, Pe, Ro, Ba], Pe, Me, Gr - one,
    [Pe, Mu, Re, Co], Me, Gr, Or - one,
    
#     [Me, Sc, Wr, Bi], [Gr, Bi],         # Sc, Wr, Co
#     [Gr, Sc, Le, Ba], Or, Pe, [Me, Le],
    
#     [Or, Pe, Re, Co], Pe, Me, Gr,   # won
    
]

like = {0:100, 1:10, 2:5, 3:0}
bag = createBag(Me, [Ki, Di, Le, Da, Ca], {Gr: 5, Or: 4, Pe: 4}) \
    >> figureKnown >> events >> processResponses >> events >> processSuggestions(_, _, like) >> events
bag >> PP;

Orchid: 330 -> 204
We know Peacock has {Or}
We know Green has {Ba}
Orchid: 204 -> 204
1344000
628250
421150
                                  Me          Gr          Or          Pe          
Green              4    S1 92%    1  -           ? 43%       ? 43%    1  ? 5%     
Mustard            3    S1 69%       -        1  -           ? 62%    1  ? 6%     
Orchid           -                   -           -           -           X        
Peacock            3    S1 49%       -           ? 43%    1  ? 6%        -        
Plum               4       127%      -           ? 43%       ? 43%       ? 40%    
Scarlet            2    S1 6%        -        1  ? 6%        -           -        
----                                                                              
Candlestick      -                   X           -           -           -        
Dagger           -                   X           -           -           -        
Lead Pipe        -                   X           -           -

In [14]:
Me = Pl
events = [
    
    [Gr, Or, Ro, Ba], Or-one,
    [Or, Or, Le, Di], Pe, [Me, Le],
    [Pe, Gr, Ro, Bi], Me, Gr-one,
#     [Or, Ro], 
    [Gr, Bi],
    [Me, Gr, Ro, Ba], Gr, [Or, Ro],
    
    [Gr, Pl, Wr, Co], Or, Pe, [Me, Wr],
    [Or, Gr, Re, Ha], Pe, Me, Gr-one,
    [Pe, Pl, Wr, Co], [Me, Wr],
    [Me, Pl, Pl, Co], [Gr, Pl],
    
    [Gr, Gr, Re, Di], Or-one,
    [Or, Gr, Re, Li], Pe, Me, Gr-one,
    [Pe, Pl, Ro, Lo], Me, Gr-one,
    [Gr, Ha],
    [Or, Di],
    [Gr, Li],
    [Me, Gr, Re, Co], Gr, Or, Pe
    
]

like = {0:100, 1:10, 2:5, 3:0}
# 60ms
bag = createBag(Me, [Wr, Le, Ca, Sc, Or], {Gr: 5, Or: 4, Pe: 4}) \
    >> figureKnown >> events >> processResponses >> events >> processSuggestions(_, _, like) >> events
bag >> PP;

We know Orchid has {Ro}
We know Green has {Bi}
We know Green has {Ha}
We know Orchid has {Di}
We know Green has {Li}
We know Green has {Pl}
6000000
21000000
17000000
                                  Me          Gr          Or          Pe          
Green            X                   -           -           -           -        
Mustard          -         109%      -           ? 16%       ? 28%       ? 64%    
Orchid           -                   X           -           -           -        
Peacock          -         109%      -           ? 16%       ? 28%       ? 64%    
Plum             -                   -           X           -           -        
Scarlet          -                   X           -           -           -        
----                                                                              
Candlestick      -                   X           -           -           -        
Dagger           -         109%      -           ? 16%       ? 28%       ? 64%    
Lead

In [15]:
Me = Pl
events = [
    
    [Pe, Pl, Ro, Ki], [Me, Ki],
    [Me, Pl, Ro, Ba], [Gr, Pl],
    [Gr, Pe, Re, Li], Or, Pe - one,
    #Or low roll
    
    [Pe, Or, Ro, Co], [Me, Co],
    [Me, Or, Ro, Bi], [Gr, Or],
    #Gr low roll
    [Or, Pe, Le, Ba], Pe, [Me, Pe],
    
#     [Pe, Mu, Le, Ba], Me, Gr, Or,
    
]

like = {0:100, 1:10, 2:5, 3:0}
bag = createBag(Me, [Ki, Ha, Co, Da, Pe], {Gr: 5, Or: 4, Pe: 4}) \
    >> figureKnown >> events >> processResponses >> events >> processSuggestions(_, _, like) >> events
bag >> PP;

Peacock: 495 -> 285
23380
23625
2048100
                                  Me          Gr          Or          Pe          
Green              4       94%       -           ? 24%       ? 38%       ? 31%    
Mustard            4       94%       -           ? 24%       ? 38%       ? 31%    
Orchid           -                   -           X           -           -        
Peacock          -                   X           -           -           -        
Plum             -                   -           X           -           -        
Scarlet            4       94%       -           ? 24%       ? 38%       ? 31%    
----                                                                              
Candlestick        4       94%       -           ? 24%       ? 38%       ? 31%    
Dagger           -                   X           -           -           -        
Lead Pipe          3    S1 30%       -           ? 24%    1  ? 6%        -        
Revolver           3    S1 61%       -        1